<h1><center>EEG Artifact Rejection Demo</center></h1>

In [1]:
%run -i utils.py

c:\users\peter\anaconda3\envs\mne\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
c:\users\peter\anaconda3\envs\mne\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


<h3><left>I. Data Wrangling</left></h3>

In [2]:
# Use the median for each of the bad channels
bad_channels_stephanie = {
    'Rew_605_rest': ['Fp1', 'Fp2'],
    'Rew_609_rest': ['F3', 'F7', 'Fp1'],
    'Rew_611_rest': ['Fp2', 'T3'],
    'Rew_613_rest': ['F7', 'Fp2'],
    'Rew_614_rest': ['C3', 'CZ', 'F3', 'F7', 'F8', 'FZ', 'Fp1', 'O1', 'P3', 'PZ', 'T3', 'T5'],
    'Rew_615_rest': ['C4', 'F4', 'F8', 'Fp2', 'O2', 'P4', 'T4', 'T6'],
    'Rew_619_rest': ['F4'],
    'Rew_622_rest': ['F7'],
    'Rew_624_rest': ['F3', 'F7', 'Fp1', 'T3', 'T4', 'T5', 'T6'],
    'Rew_626_rest': ['F3', 'F4', 'T3', 'T4', 'T5'],
    'Rew_701_rest': ['C4', 'F7', 'F8', 'O1', 'O2', 'T3', 'T3', 'T4', 'T5'],
    'Rew_702_rest': ['C3', 'F3'],
    'Rew_703_rest': ['F4', 'F7', 'F8', 'Fp2', 'T3', 'T4', 'T6'],
    'Rew_704_rest': ['C3'],
    'Rew_706_rest': ['T4']
}

In [3]:
# Load researcher and subject folders
stephanie_folder = Path("C:\\Users\\peter\\git\\EEG-artifact-rejection\\artifact-rejection\\eeg-data\\Stephanie")
sub_folders = [[file, os.path.join(stephanie_folder, file)] for file in os.listdir(stephanie_folder)]
train_sub_folders , test_sub_folders = train_test_split(sub_folders)

<h4><left>Data Architecture</left></h4>

In [4]:
subjects = dict()
for sub in train_sub_folders:
    sub_id, path_ = sub[0], sub[1]
    files = os.listdir(Path(path_))
    sub_files = dict()
    for file in files:
        full_path = os.path.join(Path(path_), file)
        if 'epoch' in file:
            sub_files['epoch'] = full_path
        if 'reject' in file:
            sub_files['reject'] = full_path
        elif 'stages' in file:
            sub_files['stage'] = full_path
    subjects[sub_id] = sub_files

<h4><left>Data Processing</left></h4>

In [5]:
epoched_dataframes = list()
for sub_ in subjects.keys():
    file_path = subjects[sub_]['epoch']
    mat_reject = subjects[sub_]['reject']
    mat_stage = subjects[sub_]['stage']

    files = load_subject_dir(file_path, mat_reject, mat_stage)
    epochs = files['epochs']
    rejects = files['rejects']

    index, scaling_time, scalings = ['epoch', 'time'], 1e3, dict(grad=1e13)
    df = epochs.to_data_frame(
        picks=None, scalings=scalings, scaling_time=scaling_time, index=index)
    df_epochs = df.groupby('epoch').mean()

    try:
        stages = files['stages']
        df_epochs['stage'] = stages
    except Exception as ex:
        print(ex)

    try:
        df_epochs = bad_channel_median(bad_channels_stephanie[sub_], df_epochs)
    except Exception as ex:
        print(ex)
    
    epoched_dataframes.append([df_epochs, rejects])

C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_626_rest\Rew_626_rest_bb_epoch.set...
6360 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_616_rest\Rew_616_rest_bb_epoch.set...
7192 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...
'Rew_616_rest'


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


...

At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_611_rest\Rew_611_rest_bb_epoch.set...
5776 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_605_rest\Rew_605_rest_bb_epoch.set...
7224 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_622_rest\Rew_622_rest_bb_epoch.set...
6372 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...


<h3><left>II. Model Selection & Training</left></h3>

In [6]:
# clfSVC = SVC(C=1.0, kernel='poly', gamma='auto_deprecated', coef0=1.0, shrinking=True, probability=False, 
#              tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', 
#              random_state=42)

In [7]:
clfSVC = SVC(C=100, cache_size=200, class_weight='balanced', coef0=0.0, decision_function_shape='ovr', degree=3, 
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, probability=False, random_state=42, shrinking=True, 
    tol=0.001, verbose=False)

In [8]:
for pair in epoched_dataframes:
    X_train, y_train = pair[0], pair[1]
    clfSVC.fit(X_train, y_train)

c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\peter\anaconda3\envs\mne\lib\site-packages\

In [9]:
from sklearn.model_selection import GridSearchCV
print("Fitting the classifier to the training set")
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['rbf', 'linear']}
clf = GridSearchCV(SVC(class_weight='balanced'), param_grid)
clf = clf.fit(X_train, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set


c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\users\peter\anaconda3\envs\mne\lib\site-packages\sklearn\svm\base.py:196: Futur

Best estimator found by grid search:
SVC(C=100, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


<h3><left>III. Model Testing & Evaluation</left></h3>

- **Precision** _(how many selected items are relevant?)_:

$$\frac{\text{True Positives}}{\text{Trust Positives + False Positives}}$$

- **Recall** _(how many relevant items are selected?)_:

$$\frac{\text{True Positives}}{\text{Trust Positives + False Negatives}}$$

- **F1 Score** _(weighted average of Precision and Recall)_:

$$\frac{2 \cdot (\text{Recall} \cdot \text{Precision})}{\text{Recall} + \text{Precision}}$$

In [10]:
test_subjects = dict()
for sub_test in test_sub_folders:
    sub_id, path_ = sub_test[0], sub_test[1]
    files = os.listdir(Path(path_))
    sub_files = dict()
    for file in files:
        full_path = os.path.join(Path(path_), file)
        if 'epoch' in file:
            sub_files['epoch'] = full_path
        if 'reject' in file:
            sub_files['reject'] = full_path
        elif 'stages' in file:
            sub_files['stage'] = full_path
    test_subjects[sub_id] = sub_files

In [11]:
epoched_dataframes_ = list()
for sub_test_ in test_subjects.keys():
    file_path = test_subjects[sub_test_]['epoch']
    mat_reject = test_subjects[sub_test_]['reject']
    mat_stage = test_subjects[sub_test_]['stage']

    files = load_subject_dir(file_path, mat_reject, mat_stage)
    epochs = files['epochs']
    rejects = files['rejects']

    index, scaling_time, scalings = ['epoch', 'time'], 1e3, dict(grad=1e13)
    df = epochs.to_data_frame(
        picks=None, scalings=scalings, scaling_time=scaling_time, index=index)
    df_epochs = df.groupby('epoch').mean()

    try:
        stages = files['stages']
        df_epochs['stage'] = stages
    except Exception as ex:
        print(ex)

    try:
        df_epochs = bad_channel_median(bad_channels_stephanie[file_path], df_epochs)
    except Exception as ex:
        print(ex)
    
    epoched_dataframes_.append([df_epochs, rejects])

C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_601_rest\Rew_601_rest_bb_epoch.set...
6476 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...
'C:\\Users\\peter\\git\\EEG-artifact-rejection\\artifact-rejection\\eeg-data\\Stephanie\\Rew_601_rest\\Rew_601_rest_bb_epoch.set'


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_623_rest\Rew_623_rest_bb_epoch.set...
7212 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...
'C:\\Users\\peter\\git\\EEG-artifact-rejection\\artifact-rejection\\eeg-data\\Stephanie\\Rew_623_rest\\Rew_623_rest_bb_epoch.set'


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


...

At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_620_rest\Rew_620_rest_bb_epoch.set...
7128 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...
'C:\\Users\\peter\\git\\EEG-artifact-rejection\\artifact-rejection\\eeg-data\\Stephanie\\Rew_620_rest\\Rew_620_rest_bb_epoch.set'


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_614_rest\Rew_614_rest_bb_epoch.set...
6364 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...
'C:\\Users\\peter\\git\\EEG-artifact-rejection\\artifact-rejection\\eeg-data\\Stephanie\\Rew_614_rest\\Rew_614_rest_bb_epoch.set'


C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\utils.py:50: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  set_file = mne.io.read_epochs_eeglab(file_path)


At least one epoch has multiple events. Only the latency of the first event will be retained.
Extracting parameters from C:\Users\peter\git\EEG-artifact-rejection\artifact-rejection\eeg-data\Stephanie\Rew_612_rest\Rew_612_rest_bb_epoch.set...
6492 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Converting time column to int64...
'C:\\Users\\peter\\git\\EEG-artifact-rejection\\artifact-rejection\\eeg-data\\Stephanie\\Rew_612_rest\\Rew_612_rest_bb_epoch.set'


In [12]:
clf_f1_score, clf_precision_score, clf_recall_score = list(), list(), list()
for pair_test in epoched_dataframes_:
    X_test, y_test = pair_test[0], pair_test[1]
    y_pred = clfSVC.predict(X_test)
    clf_precision_score += [precision_score(y_test, y_pred)]
    clf_recall_score += [recall_score(y_test, y_pred)]

In [13]:
print('Recall Scores:')    
for scr in clf_recall_score:
    print(scr)
    
print('\nPrecision Scores:')
for scr in clf_precision_score:
    print(scr)

Recall Scores:
0.6683046683046683
0.7916260954235638
0.5939849624060151
0.7823033707865169
0.683991683991684
0.8556149732620321
0.8742071881606766

Precision Scores:
0.2824506749740395
0.405688622754491
0.23140011716461628
0.2789183775663495
0.31303520456707895
0.4006677796327212
0.4210794297352342
